In [2]:
#install PySpark library (!pip install pyspark)

import pyspark

In [3]:
#pip install findspark

import findspark
findspark.init()

In [4]:
from pyspark import SparkConf, SparkContext

In [5]:
from pyspark.sql import SparkSession
spark=SparkSession.builder\
                  .master("local")\
                  .appName("WordCount")\
                  .getOrCreate()
sc=spark.sparkContext

/home/anea/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [15]:
text_de=sc.textFile("/home/anea/HTW_SS23/Prog_Alg/Spark_Project/PySpark_Project/German/Ein_Kampf_um_Rom_Felix_Dahn.txt")

In [36]:
#text_de.collect()

In [17]:
text_flat=text_de.flatMap(lambda x: x.split(' '))

In [35]:
#text_flat.collect()

In [19]:
text_flat=text_de.flatMap(lambda x: x.split(' '))\
                 .map(lambda x: (x,1))

In [34]:
#text_flat.collect()

In [21]:
text_flat=text_de.flatMap(lambda x: x.split(' '))\
                 .map(lambda x: (x,1))\
                 .reduceByKey(lambda x,y : x+y)

In [33]:
#text_flat.collect()

In [23]:
textFile = spark.read.text("/home/anea/HTW_SS23/Prog_Alg/Spark_Project/PySpark_Project/German/Ein_Kampf_um_Rom_Felix_Dahn.txt")

In [24]:
#stopwords
with open("stop_words_german.txt", "r") as f:
    text = f.read()
    stopwords = text.splitlines()

len(stopwords), stopwords[:15]

(592,
 ['a',
  'ab',
  'aber',
  'ach',
  'acht',
  'achte',
  'achten',
  'achter',
  'achtes',
  'ag',
  'alle',
  'allein',
  'allem',
  'allen',
  'aller'])

In [25]:
from pyspark.sql.functions import split

lines = textFile.select(split(textFile.value, " ").alias("line"))
lines.show(5)

+--------------------+
|                line|
+--------------------+
|       [Felix, Dahn]|
|                  []|
|[EIN, KAMPF, UM, ...|
|                  []|
|                  []|
+--------------------+
only showing top 5 rows



In [26]:
from pyspark.sql.functions import explode, col

words = lines.select(explode(col("line")).alias("word"))
words.show(15)

+----------+
|      word|
+----------+
|     Felix|
|      Dahn|
|          |
|       EIN|
|     KAMPF|
|        UM|
|       ROM|
|          |
|          |
|          |
|          |
|          |
|    ERSTES|
|     BUCH:|
|THEODERICH|
+----------+
only showing top 15 rows



In [27]:
from pyspark.sql.functions import lower

words_lower = words.select(lower(col("word")).alias("word_lower"))
words_lower.show()

+-----------+
| word_lower|
+-----------+
|      felix|
|       dahn|
|           |
|        ein|
|      kampf|
|         um|
|        rom|
|           |
|           |
|           |
|           |
|           |
|     erstes|
|      buch:|
| theoderich|
|           |
|           |
|           |
|»dietericus|
|         de|
+-----------+
only showing top 20 rows



In [28]:
from pyspark.sql.functions import regexp_extract

words_clean = words_lower.select(
    regexp_extract(col("word_lower"), "[a-z]+", 0).alias("word")
)

words_clean.show()

+----------+
|      word|
+----------+
|     felix|
|      dahn|
|          |
|       ein|
|     kampf|
|        um|
|       rom|
|          |
|          |
|          |
|          |
|          |
|    erstes|
|      buch|
|theoderich|
|          |
|          |
|          |
|dietericus|
|        de|
+----------+
only showing top 20 rows



In [29]:
words_nonull = words_clean.filter(col("word") != "")
words_nonull.show()

+----------+
|      word|
+----------+
|     felix|
|      dahn|
|       ein|
|     kampf|
|        um|
|       rom|
|    erstes|
|      buch|
|theoderich|
|dietericus|
|        de|
|     berne|
|        de|
|       quo|
|   cantant|
|   rustici|
|     usque|
|     hodie|
|    erstes|
|   kapitel|
+----------+
only showing top 20 rows



In [30]:
words_without_stopwords = words_nonull.filter(
    ~words_nonull.word.isin(stopwords))

words_count_before_removing = words_nonull.count()
words_count_after_removing = words_without_stopwords.count()

words_count_before_removing, words_count_after_removing

(315942, 133805)

In [31]:
words_count = (words_without_stopwords.groupby("word")
               .count()
               .orderBy("count", ascending=False)
               )

In [32]:
rank = 20
words_count.show(rank)

+--------+-----+
|    word|count|
+--------+-----+
|cethegus| 1066|
|   goten|  814|
|  sprach|  728|
|     ber|  717|
|    rief|  665|
|     sch|  581|
|     rom|  571|
| belisar|  555|
|     wei|  525|
|      mu|  517|
|      pr|  503|
|      st|  435|
|     gro|  434|
|witichis|  429|
|  totila|  421|
|    hand|  395|
|    teja|  393|
|   stadt|  364|
|      gl|  329|
|  narses|  311|
+--------+-----+
only showing top 20 rows

